# Acces to GDrive
These cells give access to your Google Drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import auth
auth.authenticate_user()

#Importations

In [3]:
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn import svm
from sklearn import metrics

import seaborn as sns
import matplotlib.pylab as plt

#Data extraction
These cells load the time series (from the json files) and labels (from data.csv).

In [4]:
features_filtered_climber = pd.read_csv('/content/drive/MyDrive/Boulder Project/features/150Frames_filtered_features_climber.csv')
features_filtered_success = pd.read_csv('/content/drive/MyDrive/Boulder Project/features/150Frames_filtered_features_success.csv')

In [5]:
gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1m2l52fMnXSO_SIp_LRnX-NTjt50RJhwYdJ4cCKX1xa8/edit#gid=0')
data = wb.worksheet('data').get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]

In [6]:
start = 0
end = 385
sub_df = df[start:end]
sub_df.head()

,Name,Gender (W/M),"Fail, Zone or Top (0/1/2)",Competition,Climber,Country,Boulder Label
1,c1.mp4,M,0,WC (2021),Yannick Flohé,GER,1
2,c2.mp4,M,0,WC (2021),Jan Hojer,GER,2
3,c3.mp4,M,0,WC (2021),Yannick Flohé,GER,1
4,c4.mp4,M,1,WC (2021),Jan Hojer,GER,2
5,c5.mp4,M,1,WC (2021),Jan Hojer,GER,2


In [7]:
path = '/content/drive/MyDrive/Boulder Project/'
names = sub_df["Name"].values

In [8]:
y_climber = []
y_success = []

frames = 150

for idx in range(len(names)):
  for ending in [".MOV", ".mp4"]:
    names[idx] = names[idx].replace(ending, "")
  
  path_landmarks = path + "json/" + names[idx] + "_landmarks.json"
  data = pd.read_json(path_landmarks)

  if len(data) >=frames:

    y_climber.append(sub_df['Climber'][idx+1])
    y_success.append(sub_df['Fail, Zone or Top (0/1/2)'][idx+1])

y_climber = pd.Series(data = y_climber, index = np.arange(0,329))
y_success = pd.Series(data = y_success, index = np.arange(0,329))

#Support Vector Classification 10-fold CV (using the 150 Frames Model)




**10-fold Crossvalidation: Climber**

In [9]:
k=10

keys = features_filtered_climber.keys()[1:]
kf = KFold(n_splits=k, shuffle = True, random_state = 1)

tmp_train = []
tmp_test = []
for train, test in kf.split(y_climber):
  X_train = np.array(features_filtered_climber[keys])[train]
  X_test = np.array(features_filtered_climber[keys])[test]
  Y_train = y_climber[train].values
  Y_test = y_climber[test].values
  clf_climber=svm.LinearSVC()
  clf_climber.fit(X_train,Y_train)
  y_pred_test=clf_climber.predict(X_test)
  y_pred_train=clf_climber.predict(X_train)
  tmp_train.append(metrics.accuracy_score(Y_train, y_pred_train))
  tmp_test.append(metrics.accuracy_score(Y_test, y_pred_test))

acc_train = tmp_train
acc_test = tmp_test



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

In [10]:
print('Train accuracy : ',  np.mean(acc_train))
print('Test accuracy : ', np.mean(acc_test))

Train accuracy :  0.23064996814996813
Test accuracy :  0.16070075757575758


**10-fold Crossvalidation: Achievement**

In [11]:
k = 10

keys = features_filtered_success.keys()[1:]
kf = KFold(n_splits=k, shuffle = True, random_state = 1)

tmp_train = []
tmp_test = []
for train, test in kf.split(y_success):
  X_train = np.array(features_filtered_success[keys])[train]
  X_test = np.array(features_filtered_success[keys])[test]
  Y_train = y_success[train].values
  Y_test = y_success[test].values
  clf_success=svm.LinearSVC()
  clf_success.fit(X_train,Y_train)
  y_pred_test=clf_success.predict(X_test)
  y_pred_train=clf_success.predict(X_train)
  tmp_train.append(metrics.accuracy_score(Y_train, y_pred_train))
  tmp_test.append(metrics.accuracy_score(Y_test, y_pred_test))

acc_train = np.mean(tmp_train)
acc_test = np.mean(tmp_test)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

In [12]:
print('Train accuracy : ', acc_train)
print('Test accuracy : ', acc_test)

Train accuracy :  0.5069182819182819
Test accuracy :  0.4496212121212121
